<a href="https://colab.research.google.com/github/Guilli12pm/ML_proj/blob/master/project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning project

In [1]:
!git clone https://github.com/Guilli12pm/ML_proj

Cloning into 'ML_proj'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (250/250), done.
remote: Total 280 (delta 72), reused 221 (delta 26), pack-reused 0
Receiving objects: 100% (280/280), 18.70 MiB | 29.32 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [2]:
!ls ML_proj/

check_photos.py			     haarcascade_smile.xml
create_data.py			     images.ipynb
Database			     next_ite
eyes_check.py			     project_ML.ipynb
haarcascade_eye_tree_eyeglasses.xml  __pycache__
haarcascade_eye.xml		     README.md
haarcascade_frontalface_default.xml  reset.py


In [5]:
!ls ML_proj/next_ite/

Guillaume_Loranchet


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

'1.13.1'

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
import pathlib
data_root_orig = tf.keras.utils.get_file('next_ite',
                                         'https://github.com/Guilli12pm/ML_proj/tree/master/next_ite',
                                         untar=False)
data_root = pathlib.Path(data_root_orig)
print(data_root)

  57344/Unknown - 0s 1us/step/root/.keras/datasets/next_ite


In [6]:
for item in data_root.iterdir():
  print(item)

NotADirectoryError: ignored